In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
import numpy as np
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person
from azure.cognitiveservices.vision.face.models import FaceAttributeType

In [2]:
#import vision API 
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array

In [3]:
#Retrive from Face Resource - Resource Management
faceKEY = "1ab3848ea22649299da961e8feb813a4"
faceENDPOINT = "https://face-rec-app.cognitiveservices.azure.com/"

In [4]:
# Create an authenticated FaceClient.
face_client = FaceClient(faceENDPOINT, CognitiveServicesCredentials(faceKEY))

In [5]:
#Retrive key and endpoint from Computer Vision Resource
compvision_subscription_key = "834987a3f5ef426e866617fd0739af1e"
compvision_endpoint = "https://visualpattern.cognitiveservices.azure.com/"

In [6]:
#Create an authenticated Computer Vision Client
computervision_client = ComputerVisionClient(compvision_endpoint, CognitiveServicesCredentials(compvision_subscription_key))

In [7]:
# Attributes you want returned with the API call, a list of FaceAttributeType enum (string format)
face_attributes = ['headPose', 'glasses', 'occlusion', 'blur', 'exposure', 'noise']

In [8]:
directory = r'C:\\Users\\nidhimh\\Documents\\Face\\faces' # See that i use / other way
for file in os.listdir(directory):
    print(file)

.ipynb_checkpoints
face and vision API works locally.ipynb
image_0001.jpg
image_0002.jpg
image_0003.jpg
image_0004.jpg
image_0005.jpg
image_0006.jpg
image_0007.jpg
image_0008.jpg
image_0009.jpg
image_0010.jpg
image_0011.jpg
image_0012.jpg
image_0013.jpg
image_0014.jpg
image_0015.jpg
image_0016.jpg
image_0017.jpg
image_0018.jpg
image_0019.jpg
image_0020.jpg
image_0021.jpg
image_0022.jpg
image_0023.jpg
image_0024.jpg
image_0025.jpg
image_0026.jpg
image_0027.jpg
image_0028.jpg
image_0029.jpg
image_0030.jpg
image_0031.jpg
image_0032.jpg
image_0033.jpg
image_0034.jpg
image_0035.jpg
image_0036.jpg
image_0037.jpg
image_0038.jpg
image_0039.jpg
image_0040.jpg
image_0041.jpg
image_0042.jpg
image_0043.jpg
image_0044.jpg
image_0045.jpg
image_0046.jpg
image_0047.jpg
image_0048.jpg
image_0049.jpg
image_0050.jpg
image_0051.jpg
image_0052.jpg
image_0053.jpg
image_0054.jpg
image_0055.jpg
image_0056.jpg
image_0057.jpg
image_0058.jpg
image_0059.jpg
image_0060.jpg
image_0061.jpg
image_0062.jpg
image_0063.

In [9]:
images_folder = os.path.join (os.path.dirname(os.path.abspath('')), "faces")

In [10]:
local_image_path = os.path.join (images_folder, "image_0011.jpg")

In [11]:
local_image = open(local_image_path, "rb")

In [12]:
# Detect a face with attributes, returns a list[DetectedFace]
detected_faces = face_client.face.detect_with_stream(local_image, return_face_attributes=face_attributes)

if not detected_faces:
    raise Exception(
            'No face detected from image {}'.format(os.path.basename(image)))

In [13]:
#Display the detected face with attributes and bounding box

#Face IDs are used for comparison to faces (their IDs) detected in other images.
for face in detected_faces:
    print()
    # ID of detected face
    print(face.face_id)
    # Show all facial attributes from the results
    print()
    print('Facial attributes detected:')
    print('Head pose: ', face.face_attributes.head_pose)
    print('Glasses: ', face.face_attributes.glasses)
    print('Occlusion: ')
    print('\teye_occluded: ', face.face_attributes.occlusion.eye_occluded)
    print('\tforehead_occluded: ', face.face_attributes.occlusion.forehead_occluded)
    print('Blur: ', face.face_attributes.blur)
    print('Exposure: ', face.face_attributes.exposure)
    print('Noise: ', face.face_attributes.noise)
    print()


ce231b12-0ebb-4499-9811-ecc5e456b9a8

Facial attributes detected:
Head pose:  {'additional_properties': {}, 'roll': -1.6, 'yaw': -0.9, 'pitch': -4.1}
Glasses:  GlassesType.no_glasses
Occlusion: 
	eye_occluded:  False
	forehead_occluded:  False
Blur:  {'additional_properties': {}, 'blur_level': <BlurLevel.medium: 'Medium'>, 'value': 0.3}
Exposure:  {'additional_properties': {}, 'exposure_level': <ExposureLevel.over_exposure: 'OverExposure'>, 'value': 0.77}
Noise:  {'additional_properties': {}, 'noise_level': <NoiseLevel.low: 'Low'>, 'value': 0.25}



In [14]:
# Convert width height to a point in a rectangle
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height

    return ((left, top), (right, bottom))

In [15]:
def RectCentre(faceDictionary, img):
    rect = faceDictionary.face_rectangle
    w,h=img.size
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    rightspace = w - right
    bottomspace = h - bottom
        
    if abs(left - rightspace) > 50: 
        print('Face is not horizontally centred')
    elif abs(top - bottomspace) > 75:
        print('Face is not vertically centred')
    else:
        print('Face is horizontally and vertically centred')
        

In [16]:
RectCentre(face, Image.open('image_0001.jpg'))

Face is not horizontally centred


In [17]:
# Extract the head_pose data 

In [18]:
head_pose_string = str(face.face_attributes.head_pose)

In [19]:
a, r, y, p = head_pose_string.split(',')
roll = r[8:]
roll_num = abs(float(roll))
yaw = y[7:]
yaw_num = abs(float(yaw))
pitch = p[9:-1]
pitch_num = abs(float(pitch))

In [20]:
if roll_num > 5:
    print('head is titled to the side')
elif yaw_num > 5:
    print('face is turned to the side')
elif pitch_num > 5:
    print('head is foreward or backward titled')
else:
    print('head and face is correctly positioned')

head and face is correctly positioned


In [21]:
glasses_string = str(face.face_attributes.glasses)

In [22]:
if glasses_string.__contains__("no_glasses") == False:
    print('Face is obstructed by glasses')
else:
    print('Face does not have glasses')

Face does not have glasses


In [23]:
if face.face_attributes.occlusion.eye_occluded == True:
    print('Eyes are occluded')
else:
    print('Eyes are not occluded')

Eyes are not occluded


In [24]:
if face.face_attributes.occlusion.forehead_occluded == True:
    print('Forehead is occluded')
else:
    print('Forehead is not occluded')

Forehead is not occluded


In [25]:
'''
import glob

from resize import your_resize_function

for image_file in glob.iglob('/path/to/image/dir/*.jpg'):
    your_resize_function(image_file)

'''

"\nimport glob\n\nfrom resize import your_resize_function\n\nfor image_file in glob.iglob('/path/to/image/dir/*.jpg'):\n    your_resize_function(image_file)\n\n"

In [26]:
face